---
title : "Overstromingsrisico scenarios"
execute:
    output: asis
---

Scenarios worden gebruikt om de mogelijke gevolgen van overstromingen in kaart te brengen. <br>
Hiermee kan de kans uit een fragilitycruve gecombineerd worden met de gevolgen die bepaald zijn bij een scenario om zo een beter inzicht te krijgen in het totale risico. <br>
Dit kan helpen om prioriteiten te stellen op basis van een totaal beeld en niet alleen op basis van kans of gevolg afzonderlijk. <br>
Om de scenarios te kunnen combineren moeten er een aantal stappen worden doorlopen:
- Omrekenen van dijkvakkansen naar trajectdeelkansen ([CalculateFloodScenarioProbability](#sec-CalculateFloodScenarioProbability))
- Belasting per deeltraject bepalen ([LoadFromFloodScenarioProbabilit](#sec-LoadFromFloodScenarioProbabilit))
- Met de belasting een bijpassende overstromingscenario selecteren ([SelectFloodScenarioFromLoad ](#sec-SelectFloodScenarioFromLoad))
- Van het scenario en de curve een geagregeerd risico bepalen ([CalculateFloodRisk](#sec-CalculateFloodRisk))


In [ ]:
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

In [ ]:
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "test_calculate_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.flood_scenarios import CalculateFloodScenarioProbability

## CalculateFloodScenarioProbability{sec-CalculateFloodScenarioProbability}

Dit voorbeeld laat zien hoe met behulp van de `CalculateFloodScenarioProbability`  dijkvakkansen naar trajectdeelkansen kunnen worden omgezet. <br>
Hiervoor is de **gecombineerde** faalkans per dijkvak nodig en een mapping van dijkvakken naar dijktraject delen van daar naar de overstromingsscenario gebieden.


>|⚠️ **Nog checken**|
>|---|
>- of we eerst alle mechanismes combineren, en dan naar dijktraject delen combineren (doen we nu). 
>- of eerst naar dijktraject delen combineren en dan de mechanismes combineren.

```yaml|
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    combined_failure_probability_data:
        type: csv
        file: "combined_failure_probability_data.csv"
    section_id_to_segment_id:
        type: csv
        file: "section_id_to_segment_id.csv"
    segment_failure_probability:
        type: csv
        file: "segment_failure_probability.csv"
```

In [ ]:
calculate_flood_scenario_probability = CalculateFloodScenarioProbability(
    data_adapter=data_adapter
)
calculate_flood_scenario_probability.run(
    input=["combined_failure_probability_data", "section_id_to_segment_id"],
    output=["segment_failure_probability", "gecombineerde_faalkans_dijkvakken"],
)

In [ ]:
calculate_flood_scenario_probability.df_in_sections_to_segment

In [ ]:
df = calculate_flood_scenario_probability.df_in_grouped_sections_failure_probability

In [ ]:
df["section_id"][df["section_id"] < 1.5].min()

In [ ]:
calculate_flood_scenario_probability.df_out

In [ ]:
# across all dike sections, needed for CalculateFloodRisk
calculate_flood_scenario_probability.df_out_combined_failure

## LoadFromFloodScenarioProbability{sec-LoadFromFloodScenarioProbabilit}

Nu we de faalkansen per trajectdeel hebben kunnen we met de `LoadFromFloodScenarioProbability` de belasting per trajectdeel bepalen. <br>
Dit doen we om een bijpassende overstromingsscenario te kunnen selecteren. <br>
Met de input van de vorige stap `segment_failure_probability`, maken we een koppeling naar de bijhorende bres en fragilitycurve die representatief is voor een deeltraject met de tabel `breach_id_to_segment_id`. <br>
Deze `representative_section_id_fragilitycurve` kan vervolgens uit de data van de curves in `section_failure_probability_data` worden gehaald. <br>
Met de curve wordt de hydraulische belasting bepaald voor het deeltraject.

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 48 ]

DataAdapter:
    default_options:
        csv:
            sep: ","
    segment_failure_probability:
        type: csv
        file: "segment_failure_probability.csv"
    breach_id_to_segment_id:
        type: csv
        file: "breach_id_to_segment_id.csv"
    section_failure_probability_data:
        type: csv
        file: "fragility_curve_multi_section.csv"
    flood_scenario_load_resultaten:
        type: csv
        path: "hidden_flood_scenario_load_resultaten.csv"

```

In [ ]:
config = Config(config_path=path / "test_load_from_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
data_adapter.input("segment_failure_probability")

In [ ]:
data_adapter.input("breach_id_to_segment_id")

In [ ]:
df = data_adapter.input("section_failure_probability_data")

In [ ]:
from toolbox_continu_inzicht.flood_scenarios import LoadFromFloodScenarioProbability

In [ ]:
load_from_flood_scenario_probability = LoadFromFloodScenarioProbability(
    data_adapter=data_adapter
)
load_from_flood_scenario_probability.run(
    input=[
        "segment_failure_probability",
        "breach_id_to_segment_id",
        "section_failure_probability_data",
    ],
    output="flood_scenario_load_resultaten",
)

In [ ]:
data_adapter.input("flood_scenario_load_resultaten")

In [ ]:
data_adapter.input("section_failure_probability_data")

## SelectFloodScenarioFromLoad

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 48 ]
    SelectFloodScenarioFromLoad:
        return_two_scenarios: False # default

DataAdapter:
    default_options:
        csv:
            sep: ","
    flood_scenario_loads:
        type: csv
        path: "flood_scenario_load_resultaten.csv"
    breach_location_metadata:
        type: csv
        path: "breach_location_metadata.csv"
    flood_scenario_grids:
        type: csv
        path: "flood_scenario_grids.csv"
```

In [ ]:
config = Config(config_path=path / "test_select_flood_scenario_from_load.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.flood_scenarios import SelectFloodScenarioFromLoad

In [ ]:
data_adapter.input("flood_scenario_loads")

In [ ]:
data_adapter.input("breach_location_metadata")

In [ ]:
select_flood_scenario_from_load = SelectFloodScenarioFromLoad(data_adapter=data_adapter)
select_flood_scenario_from_load.run(
    input=[
        "flood_scenario_loads",
        "breach_location_metadata",
    ],
    output="flood_scenario_grids",
)

In [ ]:
select_flood_scenario_from_load.df_out

### Keuze uit twee scenarios

Standaard wordt er per deeltraject één overstromingsscenario geselecteerd op basis van de hydraulische belasting. <br>
Met de parameter `return_two_scenarios` kan ervoor gekozen worden om twee overstromingsscenarios te selecteren. <br>
De gebruiker kan hiermee zelf bepalen welk scenario hij wil gebruiken voor de risico berekening. <br>
Of de gemiddelde schade van de twee scenario's gebruiken, of beide tonen. 

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 48 ]
    SelectFloodScenarioFromLoad:
        return_two_scenarios: True

DataAdapter:
    default_options:
        csv:
            sep: ","
    flood_scenario_loads:
        type: csv
        path: "flood_scenario_load_resultaten.csv"
    breach_location_metadata:
        type: csv
        path: "breach_location_metadata.csv"
    flood_scenario_grids:
        type: csv
        path: "flood_scenario_grids.csv"
```

In [ ]:
config = Config(
    config_path=path / "test_select_flood_scenario_from_load_two_scenarios.yaml"
)
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
select_flood_scenario_from_load = SelectFloodScenarioFromLoad(data_adapter=data_adapter)
select_flood_scenario_from_load.run(
    input=[
        "flood_scenario_loads",
        "breach_location_metadata",
    ],
    output="flood_scenario_grids",
)

In [ ]:
select_flood_scenario_from_load.df_out